# Fine-Tuning LLaMA2 for News-Aware Conversational AI

This activity explores advanced techniques for fine-tuning LLaMA-2 using Parameter-Efficient Fine-Tuning (PEFT) with the Low-Rank Adaptation (LoRA) method.

We will work with a customized version of the AGNews dataset, reformatted to match the instruction format of the Alpaca dataset. This ensures that our model is trained on a structured input format, optimized for instruction-based fine-tuning in text classification tasks.

Through supervised instruction fine-tuning, we will train our model to perform tasks specified as natural language instructions. These instructions are centered toward accurately classifying input text into one of the following categories: Business, World, Sci/Tech, and Sports.

After fine-tuning, we will evaluate the model’s conversational performance using automated benchmarks derived from another state-of-the-art language model, DeepSeek-R1-Distill-Llama-8B.

These benchmarks will measure the model’s ability to engage in meaningful and context-aware conversations, evaluating key aspects such as:

* Coherence in responses
* Relevance to user input
* Understanding of nuanced conversational cues

## Setting up the enviroment

Installs the dependent libraries for efficient training, and evaluation

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/
%pwd

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels


'/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels'

In [2]:
# Install required libraries
!pip install torch torchao transformers datasets wandb peft evaluate scikit-learn
!pip install torchtune # For Fine Tuning
!pip install ollama    # For evaluation with DeepSeek-R1
!pip install wandb     # For Logging and Reporting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

ERROR: Operation cancelled by user
^C
^C


## Get Model and Tokenizer

In [ ]:
import os

from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

# Login to Hugging Face
os.environ["HF_TOKEN"] = "<blah>"
login(token=os.environ.get("HF_TOKEN"))

from transformers.utils import logging
logging.get_logger("transformers").setLevel(logging.INFO)

# Download the baseline LLaMA-2 model https://huggingface.co/meta-llama/Llama-2-7b-hf
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/cache")
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir="/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/cache", use_safetensors=False)

print(f"Model is cached at: {model.name_or_path}")
print(f"Tokenizer is cached at: {tokenizer.name_or_path}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
loading file tokenizer.model from cache at /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/cache/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9/tokenizer.model
loading file tokenizer.json from cache at /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/cache/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9/tokenizer.json
loading file added_tokens.json from cache at None
loadin

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at meta-llama/Llama-2-7b-hf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/cache/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}



Model is cached at: meta-llama/Llama-2-7b-hf
Tokenizer is cached at: meta-llama/Llama-2-7b-hf


## Load AGNews Dataset

In [2]:
%cd /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/

/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels


In [ ]:
from datasets import load_dataset

dataset = load_dataset("ag_news")
print(f"Dataset loaded: {len(dataset['train'])} training examples")

# View a sample
sample = dataset['train'][0]

print(f"Text: {sample['text']}")
print(f"Label: {sample['label']}")

Dataset loaded: 120000 training examples
Text: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
Label: 2


## Prepare and Persist the Dataset

Preprocess dataset to be in the instruction-tuning format of Alpaca dataset

In [4]:
%pwd

'/content/drive/MyDrive/Colab Notebooks/CMU_LargeLanguageModels'

In [5]:
from collections import Counter
from datasets import DatasetDict

# Get label names for mapping
label_names = dataset['train'].features['label'].names

# Count occurrences of each label
label_counts = Counter(dataset['train']['label'])

# Print unique labels with counts
for label_id, count in label_counts.items():
    print(f"{label_names[label_id]}: {count}")

# Prepare dataset with TorchTune format
def preprocess_data(examples):
    return {
        "instruction": ["Classify this news article."] * len(examples['text']),  # Repeating instruction
        "input": examples['text'],
        "output": [label_names[label] for label in examples['label']]
    }

processed_train = dataset["train"].map(preprocess_data, batched=True)
processed_test = dataset["test"].map(preprocess_data, batched=True)

processed_train = processed_train.remove_columns(["text", "label"])
processed_test = processed_test.remove_columns(["text", "label"])

# Save preprocessed dataset
preprocessed_dataset = DatasetDict({
    "train": processed_train,
    "test": processed_test
})

# Persist preprocessed dataset
save_path = "/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/preprocessed_agnews"
preprocessed_dataset.save_to_disk(save_path)

print("Processed Train Dataset features: ", processed_train.features)
print("Processed Test Dataset features: ", processed_test.features)

print(f"Processed Train Dataset: {len(processed_train)} examples")
print(f"Processed Test Dataset: {len(processed_test)} examples")

print(f"Sample Processed Train Dataset: {processed_train[0]}")
print(f"Sample Processed Test Dataset: {processed_test[10]}")

Business: 30000
Sci/Tech: 30000
Sports: 30000
World: 30000


Saving the dataset (0/1 shards):   0%|          | 0/120000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7600 [00:00<?, ? examples/s]

Processed Train Dataset features:  {'instruction': Value(dtype='string', id=None), 'input': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None)}
Processed Test Dataset features:  {'instruction': Value(dtype='string', id=None), 'input': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None)}
Processed Train Dataset: 120000 examples
Processed Test Dataset: 7600 examples
Sample Processed Train Dataset: {'instruction': 'Classify this news article.', 'input': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'output': 'Business'}
Sample Processed Test Dataset: {'instruction': 'Classify this news article.', 'input': 'Group to Propose New High-Speed Wireless Format  LOS ANGELES (Reuters) - A group of technology companies  including Texas Instruments Inc. &lt;TXN.N&gt;, STMicroelectronics  &lt;STM.PA&gt; and Broadcom Corp. &lt;BRCM.O&gt;, on Thursday said th

## Load the Preprocessed Dataset

In [6]:
%pwd

'/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels'

In [7]:
from datasets import load_from_disk
import json

# Load preprocessed dataset from saved location
dataset_path = "/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/preprocessed_agnews"
dataset = load_from_disk(dataset_path)

print(f"Loaded preprocessed dataset with (train, test) samples: ({len(dataset['train'])} , {len(dataset['test'])})")

#dataset = dataset["train"]
#print(dataset[0])


# Convert train to JSON Lines format
train_data = dataset["train"]
with open("/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/agnews_train.jsonl", "w") as f:
    for item in train_data:
        f.write(json.dumps(item) + "\n")

print(f"Saved {len(train_data)} examples to JSONL file")

test_data = dataset["test"]
with open("/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/agnews_test.jsonl", "w") as f:
    for item in train_data:
        f.write(json.dumps(item) + "\n")

print(f"Saved {len(test_data)} examples to JSONL file")


Loaded preprocessed dataset with (train, test) samples: (120000 , 7600)
Saved 120000 examples to JSONL file
Saved 7600 examples to JSONL file


## FineTuning with LoRA using TorchTune

**TorchTune** is a PyTorch library for LLM fine-tuning that prioritizes simplicity, correctness, and accessibility. It's designed to work seamlessly with PyTorch while making LLM experimentation accessible to everyone.

**TorchTune Recipes**

Recipes are the primary entry points for torchtune users. These can be thought of as hackable, singularly-focused scripts for interacting with LLMs including fine-tuning, inference, evaluation, and quantization.

**LoRA** is a Parameter-efficient fine-tuning technique (PEFT)

### Full List of ALL Recipes

In [15]:
!tune ls

RECIPE                                   CONFIG                                  
full_finetune_single_device              llama2/7B_full_low_memory               
                                         code_llama2/7B_full_low_memory          
                                         llama3/8B_full_single_device            
                                         llama3_1/8B_full_single_device          
                                         llama3_2/1B_full_single_device          
                                         llama3_2/3B_full_single_device          
                                         mistral/7B_full_low_memory              
                                         phi3/mini_full_low_memory               
                                         qwen2/7B_full_single_device             
                                         qwen2/0.5B_full_single_device           
                                         qwen2/1.5B_full_single_device           
                

### Customizing configs using tune cp

There are 2 ways to customize recipe configs:

* Using the tune cp command to copy a config from the torchtune library , modify it, and then use it when running the recipe.
* Specifying the changed config values in a key=value format on the command line when running the recipe. Used this method for clarity.



### Get Model and Tokenizer for use with TorchTune CLI

This is an alternate approach that uses TorchTune CLI over the use of the HuggingFace transformer library

In [13]:
# This is not needed as the model and tokenizer are already present in the cache_dir
#!tune download meta-llama/Llama-2-7b-hf --output-dir ./llama2-7b

### Get a copy of Llama2 configs for TorchTune recipe

Lookup corresponding Config for Recipe named -  lora_finetune_single_device, from tune ls

In [8]:
# Copy llama2 config for recipe named lora_finetune_single_device
# (recipe_name : config_names from tune ls)
!tune cp llama2/7B_lora_single_device "/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/llama2_7B_lora_single_device.yaml"


Copied file to /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/llama2_7B_lora_single_device.yaml


### Make output directories for logging and model outputs

In [9]:
%cd /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/

/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels


In [10]:
!mkdir -p llama2_7B_lora_single_device_outputs
!mkdir -p llama2_7B_lora_single_device_outputs/wandb_logs

### FineTune with LoRA using TorchTune CLI

Here we use the default configs and specify changes in the command line. Alternately, we could also make changes locally and specify path to the modified config.

We train on 70% of the training data with a batch size of 2 and log to wandb.

In [11]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 7600
    })
})


In [12]:
from torchtune.datasets import alpaca_cleaned_dataset

# Define additional arguments for `load_dataset`
load_dataset_kwargs = {
    "split": "train",  # Load only 10% of the training data
    "data_files": "/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/agnews_train.jsonl"
}
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/cache/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9"
)
# Load the dataset
dataset = alpaca_cleaned_dataset(source="json", tokenizer=tokenizer, **load_dataset_kwargs)


loading file tokenizer.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import os
os.environ["WANDB_API_KEY"] = "<blah>"

In [ ]:
!tune run lora_finetune_single_device \
        --config llama2/7B_lora_single_device \
        output_dir="/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/llama2_7B_lora_single_device_outputs" \
        checkpointer.checkpoint_dir="/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/cache/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9/" \
        tokenizer.path="/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/cache/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9/tokenizer.model" \
        dataset._component_=torchtune.datasets.alpaca_cleaned_dataset \
        dataset.source="json" \
        dataset.column_map='{"instruction": "instruction", "input": "input", "output": "output"}' \
        dataset.data_files="/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/agnews_train.jsonl" \
        dataset.split=train[:80%] \
        dataset.train_on_input=False \
        lr_scheduler.num_warmup_steps=5 \
        batch_size=2 \
        gradient_accumulation_steps=8 \
        metric_logger._component_=torchtune.training.metric_logging.WandBLogger \
        metric_logger.project=llama2_finetune_agnews_with_cli  metric_logger.group=llama2_7b_lora_batch_8 \
        metric_logger.job_type=lora_single_device \
        metric_logger.log_dir="/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/llama2_7B_lora_single_device_outputs/wandb_logs" \
        log_every_n_steps=1 \
        log_peak_memory_stats=True


INFO:torchtune.utils._logging:Running LoRAFinetuneRecipeSingleDevice with resolved config:

batch_size: 2
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  adapter_checkpoint: null
  checkpoint_dir: /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/cache/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9/
  checkpoint_files:
  - pytorch_model-00001-of-00002.bin
  - pytorch_model-00002-of-00002.bin
  model_type: LLAMA2
  output_dir: /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/llama2_7B_lora_single_device_outputs
  recipe_checkpoint: null
compile: false
dataset:
  _component_: torchtune.datasets.alpaca_cleaned_dataset
  column_map:
    input: input
    instruction: instruction
    output: output
  data_files: /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/agnews_train.jsonl
  packed: false
  source: json
  split: train[:80%]
  train_on_input: false
device: cuda
dtype: bf16
enable_

### Observe fine tuned model v.s. base model

In [1]:
# Fine tuned model and LoRA adapter checkpoints
%ls -lh "/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/llama2_7B_lora_single_device_outputs/epoch_0"

total 3.9G
-rw------- 1 root root  134 Feb 27 01:21 adapter_config.json
-rw------- 1 root root  35M Feb 27 01:21 adapter_model.pt
-rw------- 1 root root  35M Feb 27 01:21 adapter_model.safetensors
-rw------- 1 root root  609 Feb 25 05:29 config.json
-rw------- 1 root root 3.8G Feb 27 08:09 ft-model-00001-of-00002.safetensors
-rw------- 1 root root  188 Feb 25 05:44 generation_config.json
-rw------- 1 root root  28K Feb 27 01:21 model.safetensors.index.json
-rw------- 1 root root  414 Feb 25 05:29 special_tokens_map.json
-rw------- 1 root root  776 Feb 25 05:29 tokenizer_config.json
-rw------- 1 root root 1.8M Feb 25 05:29 tokenizer.json
-rw------- 1 root root 489K Feb 25 05:29 tokenizer.model
-rw------- 1 root root 2.6K Feb 26 17:32 torchtune_config.yaml


In [2]:
# Base Model without fine tuning
%ls -lh "/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/cache/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9/"


total 9.0K
lrw------- 1 root root   52 Feb 25 05:29 config.json -> ../../blobs/34f901200fa131819b355bc4bed876c957a77a5a
lrw------- 1 root root   52 Feb 25 05:44 generation_config.json -> ../../blobs/aa1b3d3486df56a0699ce90c33283b13556fb5a3
lrw------- 1 root root   76 Feb 25 05:30 model-00001-of-00002.safetensors -> ../../blobs/4ec71fd53e99766de38f24753b30c9e8942630e9e576a1ba27b0ec531e87be41
lrw------- 1 root root   76 Feb 25 05:30 model-00002-of-00002.safetensors -> ../../blobs/41780b5dac322ac35598737e99208d90bdc632a1ba3389ebedbb46a1d8385a7f
lrw------- 1 root root   52 Feb 25 05:29 model.safetensors.index.json -> ../../blobs/8b6245796e966e50960a317e4a54aa7bf73b0186
lrw------- 1 root root   76 Feb 25 06:16 pytorch_model-00001-of-00002.bin -> ../../blobs/ee62ed2ad7ded505ae47df50bc6c52916860dfb1c009df4715148cc4bfb50d2f
lrw------- 1 root root   76 Feb 25 06:17 pytorch_model-00002-of-00002.bin -> ../../blobs/1fd7762035b3ca4f2d6af6bf10129689a119b7c38058025f9842511532ea02fb
lrw------- 1 root 

## Run Inference


#### Inference with Baseline Model

See *model_inference.py*

In [1]:
%cd /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/

/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels


### Inference with the Fine Tuned Model using PEFT from [HuggingFace library](https://github.com/huggingface/peft)

See *model_inference_py*

In [ ]:
!python model_inference.py

2025-02-28 05:06:08.533072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740719168.834962    3338 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740719168.914968    3338 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-28 05:06:09.540321: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Loading baseline model: meta-llama/Llama-2-7b-hf from cache_path: /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageMo

## Evaluating the fine-tuned LLM

Fine-tuned LLMs can be evaluated through multiple approaches:
* With Short-answer and multiple-choice benchmarks such as Measuring Massive Mutitask Language Understanding (MMLU to test the general knowledge of a model)
* With Human preference comparison to other LLMs
* With Automated conversational benchmarks, where another LLM is used to evaluate the responses.

Here, we are primarily interested in assessing conversational performance rather than just the ability to answer multiple-choice questions, and so human evaluation and automated metrics are more relevant.

We use the method here to automate the response evaluation of the fine-tuned Llama2 using another, larger LLM - DeepSeek-R1-Distill-Llama-8B.

We will use Ollama to run this model locally as shown below.

**DeepSeek-R1-Distill-Llama-8B**

`ollama run deepseek-r1:8b`


In [2]:
!ollama run deepseek-r1:8b

/bin/bash: line 1: ollama: command not found


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels/

/content/drive/My Drive/Colab Notebooks/CMU_LargeLanguageModels


In [ ]:
!python ollama_evaluate.py --file_path="./agnews_test_response.jsonl"